# Water mass transformation code
Adapted from Elizabeth Maroon

To calculate the WMT due to any other single process, read in any diag that contributes a freshwater flux, heat flux, or salt flux. It should be possible to fully decompose SHF and SFWF into different contributors. 

varsneeded for freshwater part => ['SFWF', 'EVAP_F', 'PREC_F', 'IOFF_F', 'SNOW_F', 'ROFF_F', 'SALT_F', 'MELT_F']   
varsneeded for heat part => ['SHF', 'QFLUX', 'LWDN_F', 'LWUP_F', 'SENH_F', 'SHF_QSW', 'MELTH_F'] 

Freshwater fluxes get treated the same as SFWF, while heat fluxes get treated the same as SHF. Salt fluxes (brine rejection and frazil) need to be scaled to a practical salinity and then get multiplied by beta to convert them to a surface density flux.  

NOTE: Think about any other possible sources of transformation not included above. Estuary Box Model? Robert filter? Subscale grid mixing?

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pop_tools
from datetime import timedelta
import glob
import dask
from matplotlib.gridspec import GridSpec

In [ ]:
# ONLY RUN THIS CELL ONCE
# NOTE: you may need to change the project number
from dask_jobqueue import PBSCluster
from dask.distributed import Client
#put your projection number here
proj = 'P93300065'

# Setup your PBSCluster
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory='10GB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=1:mem=10GB', # Specify resources
    project=proj, # Input your project ID here
    walltime='02:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
#cluster.scale(18)
cluster.adapt(minimum=1,maximum=10,wait_count=60)

In [ ]:
# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [ ]:
client #.restart() #when needing to refresh stuff

In [ ]:
#choose cases and data paths
case1 = 'b.e21.B1850.f09_g17.CMIP6-piControl.001b'
case2 = 'b.e21.B1850.f09_g17.CMIP6-piControl.001_bl99b'
label1 = 'MUSHY'
label2 = 'BL99'

data_dir1 = '/glade/campaign/cgd/ppc/dbailey/'+case1+'/ocn/proc/tseries/month_1/'
data_dir2 = '/glade/campaign/cgd/ppc/dbailey/'+case2+'/ocn/proc/tseries/month_1/'

#to calculate surface density sigma_2
varsneeded = ['SALT', 'TEMP'] 
#vars needed for FW pieces
varsneeded = varsneeded + ['SFWF', 'EVAP_F', 'PREC_F', 'IOFF_F', 'SNOW_F', 'ROFF_F', 'SALT_F', 'MELT_F']   
#and vars needed for heat pieces
varsneeded = varsneeded + ['SHF', 'QFLUX', 'LWDN_F', 'LWUP_F', 'SENH_F', 'SHF_QSW', 'MELTH_F']  

In [ ]:
%%time
#reading in files
ds1 = []
ds2 = []
for varname in varsneeded:
    print("loading "+varname)   
    # This loop takes a long time
    my_files=sorted(glob.glob(data_dir1+case1+'.pop.h.'+varname+'.*.nc'))[-2:]
    ds1.append(xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal'))
    my_files=sorted(glob.glob(data_dir2+case2+'.pop.h.'+varname+'.*.nc'))[-2:]
    ds2.append(xr.open_mfdataset(my_files,combine='by_coords',chunks={}, parallel=True, compat='override', coords='minimal'))
    
    # TEST WITH SINGLE FILE OPENING
    #ds1.append(xr.open_dataset(data_dir1+case1+'.pop.h.'+varname+'.092101-097012.nc'))
    #ds2.append(xr.open_dataset(data_dir2+case2+'.pop.h.'+varname+'.092101-097012.nc'))

print("Merging files now")
ds1=xr.merge(ds1,compat='override')
ds2=xr.merge(ds2,compat='override')

## Just coastal points for WMT

In [ ]:
# Load in coastal mask
ds_masks = xr.open_mfdataset('/glade/p/cgd/ppc/duvivier/cesm2_antarctic_polynya/DATA/masks/antarctic_ocean_masks_2.nc')
my_mask = ds_masks['coast_mask']

In [ ]:
my_mask.plot()

In [ ]:
#region to perform WMT over - coastal mask only 
ds1=ds1.isel(z_t=0).where(my_mask==1)
ds2=ds2.isel(z_t=0).where(my_mask==1)

#test that masking worked properly
ds1["TEMP"].isel(time=0).plot()

In [ ]:
%%time
# Takes ~8 min (only a few sec with dask)
ds1.persist();#.load(); 
ds2.persist();#.load();

#added a dask cluster+persist to make this go faster behind the scenes.

In [ ]:
# set some values
s0=ds1['ocn_ref_salinity']
s0 = s0[0]
cp=ds1['cp_sw'].values/10000  #from erg/g/K -> J/kgK
cp = cp[0]
fusion=ds1['latent_heat_fusion'].values/10000 #from erg/g -> J/kg
fusion = fusion[0]

# get grid information
tlat=ds1['TLAT']
tlon=ds1['TLONG']
tarea=ds1['TAREA']/(100*100)  # convert to m2
depth=ds1['z_t']
ht=ds1['HT']
depth_top=depth.where(depth<ht, np.nan)/100

# make a quick plot of the area the WMT is done over
plt.pcolormesh(depth_top.isel(time=0))
plt.colorbar()

### Load Data from CASE1

In [ ]:
## CASE1
#had to add loads to get everything on head node b/c eos method not working with dask
salt1=ds1['SALT'].load() 
temp1=ds1['TEMP'].load()

#get stuff from POP eqn of state (EOS), note that we're now refferencing pressure array to 2000m (i.e. sigma_2)
press_arr = xr.DataArray(2000*np.ones(depth_top.shape),dims=depth_top.dims, coords=depth_top.coords)
rho1,drhods1,drhodt1=pop_tools.eos(salt=salt1,temp=temp1,return_coefs=True,pressure=press_arr)

#construct alpha and beta using POP methods
alpha1=-1*drhodt1/rho1
beta1=drhods1/rho1

In [ ]:
#adjusting SHF to make for total heat part, adding QFLUX, then dividing by cp dw
shf_scaled1 = (ds1['SHF']+ds1['QFLUX'])/cp

#adjusting FW to remove the frazil part, then multiplying to turn FW flux into salinity flux
fw_scaled1 = (ds1['SFWF'] - ds1['QFLUX']/fusion)*(salt1/1000)/(1-salt1/1000)

In [ ]:
#multiply by alpha and beta to get density fluxes
heatpart1=-alpha1*shf_scaled1
fwpart1=-1*fw_scaled1*beta1

#combine to get total boundary forced density flux
densflux1=heatpart1+fwpart1

In [ ]:
#Calculate components of the FW part

#salt flux (brine rejection) part
#needs to be scaled from kg salt to a FW flux
#then it can be multiplied by beta to convert to a surface density flux
salinity_factor1 = ds1['salinity_factor']
sflux_factor1 = ds1['sflux_factor']            
salt_f_fixed1 = ds1['SALT_F']*sflux_factor1/salinity_factor1   #doesn't need to be multiplied by -1 b/c -1 in salinity_factor
saltpart1 = -1*beta1*salt_f_fixed1 *(salt1/1000)/(1-salt1/1000)  

#freshwater flux from qflux (frazil) part
qflux_fw_fixed1 = -1*ds1['QFLUX']/fusion    #releasing heat of fusion = positive ocean heating = removal of FW -> multiply by -1 
frazilpart1=-1*beta1*qflux_fw_fixed1*(salt1/1000)/(1-salt1/1000)

#These are FW fluxes
#evaporation part  
evappart1 = -1*beta1*(ds1['EVAP_F']*(salt1/1000)/(1-salt1/1000))

#ice melt part
meltpart1 = -1*beta1*(ds1['MELT_F']*(salt1/1000)/(1-salt1/1000))

#precip part
precippart1 = -1*beta1*(ds1['PREC_F']*(salt1/1000)/(1-salt1/1000))

#runoff part - sum of liquid and solid runoff
runoffpart1 = -1*beta1*((ds1['ROFF_F']+ds1['IOFF_F'])*(salt1/1000)/(1-salt1/1000))

#total salt reconstructed
fwpart_recons1 = saltpart1+frazilpart1+evappart1+meltpart1+precippart1+runoffpart1

In [ ]:
# heat components - not going to get into these because heat flux is so much smaller than FW fluxes

##to get latent heat flux, need to convert EVAP_F from FW flux to heat flux via latent heat of vaporization
#vaporization = ds['latent_heat_vapor']

#latent_heat = ds['EVAP_F']*vaporization
##can now be multiplied by alpha to convert to a surf dens flux

##snow melt needs to be converted from a FW flux to a heat flux via latent heat of fusion
#snow_melt_heating = ds['SNOW_F']*fusion*-1
##Multiplied by negative one b/c melt (positive) leads to cooling (negative)
##can now be multiplied by alpha to convert to a surf dens flux

##same goes for ice melt runoff (solid)
#ioff_heating = ds['IOFF_F']*fusion*-1

### Load Data from CASE2

In [ ]:
## CASE2
#had to add loads b/c eos method not working with dask
salt2=ds2['SALT'].load()
temp2=ds2['TEMP'].load()
depth_top.load()

#get stuff from POP EOS
# note that we already defined the pressure=press_arr to reference 2000 m above, when loading case 1
rho2,drhods2,drhodt2=pop_tools.eos(salt=salt2,temp=temp2,return_coefs=True, pressure=press_arr)

#construct alpha and beta using POP methods
alpha2=-1*drhodt2/rho2
beta2=drhods2/rho2

In [ ]:
#adjusting SHF to make for total heat part, adding QFLUX, then dividing by cp dw
shf_scaled2 = (ds2['SHF']+ds2['QFLUX'])/cp

#adjusting FW to remove the frazil part, then multiplying to turn salt flux into FW flux
fw_scaled2 = (ds2['SFWF'] - ds2['QFLUX']/fusion)*(salt2/1000)/(1-salt2/1000)

In [ ]:
#multiply by alpha and beta to get density fluxes
heatpart2=-alpha2*shf_scaled2
fwpart2=-1*fw_scaled2*beta2

#combine to get total boundary forced density flux
densflux2=heatpart2+fwpart2

In [ ]:
#Calculate components of the freshwater flux

#salt flux (brine rejection) part
#needs to be scaled from kg salt to a practical salinity
#then it can be multiplied by beta to convert to a surface density flux
salinity_factor2 = ds2['salinity_factor']
sflux_factor2 = ds2['sflux_factor']            
salt_f_fixed2 = ds2['SALT_F']*sflux_factor2/salinity_factor2           #doesn't need to be multiplied by -1 b/c -1 in salinity_factor
saltpart2=-1*beta2*salt_f_fixed2*(salt2/1000)/(1-salt2/1000)  

#freshwater flux from qflux (frazil) part
qflux_fw_fixed2 = -1*ds2['QFLUX']/fusion                               #releasing heat of fusion = positive ocean heating = removal of FW -> multiply by -1 
frazilpart2= -1*beta2*qflux_fw_fixed2*(salt2/1000)/(1-salt2/1000)

#These are FW fluxes
#evaporation part
evappart2 = -1*beta2*(ds2['EVAP_F']*(salt2/1000)/(1-salt2/1000))

#ice melt part
meltpart2 = -1*beta2*(ds2['MELT_F']*(salt2/1000)/(1-salt2/1000))

#precip part
precippart2 = -1*beta2*(ds2['PREC_F']*(salt2/1000)/(1-salt2/1000))

#runoff part - sum of liquid and solid runoff
runoffpart2 = -1*beta2*((ds2['ROFF_F']+ds2['IOFF_F'])*(salt2/1000)/(1-salt2/1000))

#total salt reconstructed
fwpart_recons2 = saltpart2+frazilpart2+evappart2+meltpart2+precippart2+runoffpart2

### Calculate WMT

In [ ]:
#function for WMT 
def wmtrans(oneflux,dens_class,rho):
    binsize = dens_class[2]-dens_class[1]
    #assumes uniform density class spacing
    wmt=[]
    for ii in range(len(dens_class[0:-1])):
        densf_byclass = oneflux.where((rho>=dens_class[ii]) & (rho<dens_class[ii+1]),np.nan)
        wmt.append((densf_byclass*tarea).sum(['nlat','nlon'])/binsize)
    wmt = xr.concat(wmt, dim = sigma)
    return wmt
    
#set up density classes
binsize = 0.1 #typical value used
dens_class = np.arange(np.floor(rho1.min()),np.ceil(rho1.max()),binsize)
sigma = xr.DataArray(dens_class[0:-1]+binsize/2-1000, dims=['sigma'], coords={'sigma':dens_class[0:-1]+binsize/2-1000})


In [ ]:
%%time
# Takes ~1 min
# CASE1
#calculate the WMT
wmt1 = wmtrans(densflux1, dens_class, rho1).load()
wmt1_heat = wmtrans(heatpart1, dens_class, rho1).load()
wmt1_fresh = wmtrans(fwpart1, dens_class, rho1).load()
#added loads to make plotting be faster later

#calculate WMF from WMT   
wmf1 = - wmt1.diff('sigma')#/binsize
wmf1 = wmf1.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_fresh = - wmt1_fresh.diff('sigma')#/binsize
wmf1_fresh = wmf1_fresh.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_heat = - wmt1_heat.diff('sigma')#/binsize
wmf1_heat = wmf1_heat.assign_coords({'sigma':sigma[0:-1]+binsize/2})

#got rid of division by binsize to keep units = to Sv and to follow Abernathey convention

In [ ]:
%%time
# Takes ~1 min
# CASE2
#calculate the WMT
wmt2 = wmtrans(densflux2, dens_class, rho2).load()
wmt2_heat = wmtrans(heatpart2, dens_class, rho2).load()
wmt2_fresh = wmtrans(fwpart2, dens_class, rho2).load()
# calculate WMF from WMT  
wmf2 = - wmt2.diff('sigma')#/binsize
wmf2 = wmf2.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_fresh = - wmt2_fresh.diff('sigma')#/binsize
wmf2_fresh = wmf2_fresh.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_heat = - wmt2_heat.diff('sigma')#/binsize
wmf2_heat = wmf2_heat.assign_coords({'sigma':sigma[0:-1]+binsize/2})

#got rid of division by binsize to keep units = to Sv and to follow Abernathey convention

### Calculate WMT - individual FW terms

In [ ]:
%%time
# ~1min
# CASE1 - FW terms
#calculate the WMT
wmt1_salt = wmtrans(saltpart1, dens_class, rho1).load()
wmt1_frazil = wmtrans(frazilpart1, dens_class, rho1).load()
wmt1_evap = wmtrans(evappart1, dens_class, rho1).load()
wmt1_melt = wmtrans(meltpart1, dens_class, rho1).load()
wmt1_precip = wmtrans(precippart1, dens_class, rho1).load()
wmt1_runoff = wmtrans(runoffpart1, dens_class, rho1).load()
wmt1_recons = wmtrans(fwpart_recons1, dens_class, rho1).load()

#calculate WMF from WMT   
wmf1_salt = - wmt1_salt.diff('sigma')#/binsize
wmf1_salt = wmf1_salt.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_frazil = - wmt1_frazil.diff('sigma')#/binsize
wmf1_frazil = wmf1_frazil.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_evap = - wmt1_evap.diff('sigma')#/binsize
wmf1_evap = wmf1_evap.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_precip = - wmt1_precip.diff('sigma')#/binsize
wmf1_precip = wmf1_precip.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_runoff = - wmt1_runoff.diff('sigma')#/binsize
wmf1_runoff = wmf1_runoff.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_melt = - wmt1_melt.diff('sigma')#/binsize
wmf1_melt = wmf1_melt.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf1_recons = - wmt1_recons.diff('sigma')#/binsize
wmf1_recons = wmf1_recons.assign_coords({'sigma':sigma[0:-1]+binsize/2})

#loaded WMTs and got rid of division by binsize for WMF

In [ ]:
%%time
# ~1min
# CASE2 - FW terms
#calculate the WMT
wmt2_salt = wmtrans(saltpart2, dens_class, rho2).load()
wmt2_frazil = wmtrans(frazilpart2, dens_class, rho2).load()
wmt2_evap = wmtrans(evappart2, dens_class, rho2).load()
wmt2_melt = wmtrans(meltpart2, dens_class, rho2).load()
wmt2_precip = wmtrans(precippart2, dens_class, rho2).load()
wmt2_runoff = wmtrans(runoffpart2, dens_class, rho2).load()
wmt2_recons = wmtrans(fwpart_recons2, dens_class, rho2).load()

#calculate WMF from WMT   
wmf2_salt = - wmt2_salt.diff('sigma')#/binsize
wmf2_salt = wmf2_salt.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_frazil = - wmt2_frazil.diff('sigma')#/binsize
wmf2_frazil = wmf2_frazil.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_evap = - wmt2_evap.diff('sigma')#/binsize
wmf2_evap = wmf2_evap.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_precip = - wmt2_precip.diff('sigma')#/binsize
wmf2_precip = wmf2_precip.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_runoff = - wmt2_runoff.diff('sigma')#/binsize
wmf2_runoff = wmf2_runoff.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_melt = - wmt2_melt.diff('sigma')#/binsize
wmf2_melt = wmf2_melt.assign_coords({'sigma':sigma[0:-1]+binsize/2})
wmf2_recons = - wmt2_recons.diff('sigma')#/binsize
wmf2_recons = wmf2_recons.assign_coords({'sigma':sigma[0:-1]+binsize/2})

#loading WMTs and not dividing by binsize

### Calculate means and differences

In [ ]:
# Get annual mean WMT values 
# CASE1
wmt1_ann = wmt1.mean(dim="time")
wmf1_ann = wmf1.mean(dim="time")
wmt1_fresh_ann = wmt1_fresh.mean(dim="time")
wmf1_fresh_ann = wmf1_fresh.mean(dim="time")
wmt1_heat_ann = wmt1_heat.mean(dim="time")
wmf1_heat_ann = wmf1_heat.mean(dim="time")
wmt1_salt_ann = wmt1_salt.mean(dim="time")
wmf1_salt_ann = wmf1_salt.mean(dim="time")
wmt1_frazil_ann = wmt1_frazil.mean(dim="time")
wmf1_frazil_ann = wmf1_frazil.mean(dim="time")
wmt1_evap_ann = wmt1_evap.mean(dim="time")
wmf1_evap_ann = wmf1_evap.mean(dim="time")
wmt1_precip_ann = wmt1_precip.mean(dim="time")
wmf1_precip_ann = wmf1_precip.mean(dim="time")
wmt1_runoff_ann = wmt1_runoff.mean(dim="time")
wmf1_runoff_ann = wmf1_runoff.mean(dim="time")
wmt1_melt_ann = wmt1_melt.mean(dim="time")
wmf1_melt_ann = wmf1_melt.mean(dim="time")
wmt1_recons_ann = wmt1_recons.mean(dim="time")
wmf1_recons_ann = wmf1_recons.mean(dim="time")

In [ ]:
# Get annual mean WMT values 
# CASE2
wmt2_ann = wmt2.mean(dim="time")
wmf2_ann = wmf2.mean(dim="time")
wmt2_fresh_ann = wmt2_fresh.mean(dim="time")
wmf2_fresh_ann = wmf2_fresh.mean(dim="time")
wmt2_heat_ann = wmt2_heat.mean(dim="time")
wmf2_heat_ann = wmf2_heat.mean(dim="time")
wmt2_salt_ann = wmt2_salt.mean(dim="time")
wmf2_salt_ann = wmf2_salt.mean(dim="time")
wmt2_frazil_ann = wmt2_frazil.mean(dim="time")
wmf2_frazil_ann = wmf2_frazil.mean(dim="time")
wmt2_evap_ann = wmt2_evap.mean(dim="time")
wmf2_evap_ann = wmf2_evap.mean(dim="time")
wmt2_precip_ann = wmt2_precip.mean(dim="time")
wmf2_precip_ann = wmf2_precip.mean(dim="time")
wmt2_runoff_ann = wmt2_runoff.mean(dim="time")
wmf2_runoff_ann = wmf2_runoff.mean(dim="time")
wmt2_melt_ann = wmt2_melt.mean(dim="time")
wmf2_melt_ann = wmf2_melt.mean(dim="time")
wmt2_recons_ann = wmt2_recons.mean(dim="time")
wmf2_recons_ann = wmf2_recons.mean(dim="time")

In [ ]:
# calculate differences between two datasets
wmt_ann_diff = wmt1_ann - wmt2_ann
wmf_ann_diff = wmf1_ann - wmf2_ann

wmt_fresh_ann_diff = wmt1_fresh_ann - wmt2_fresh_ann
wmf_fresh_ann_diff = wmf1_fresh_ann - wmf2_fresh_ann

wmt_heat_ann_diff = wmt1_heat_ann - wmt2_heat_ann
wmf_heat_ann_diff = wmf1_heat_ann - wmf2_heat_ann

#E-P-R
wmt_epr_ann_diff = (wmt1_evap_ann+wmt1_precip_ann+wmt1_runoff_ann) - \
                   (wmt2_evap_ann+wmt2_precip_ann+wmt2_runoff_ann)
wmf_epr_ann_diff = (wmf1_evap_ann+wmf1_precip_ann+wmf1_runoff_ann) - \
                   (wmf2_evap_ann+wmf2_precip_ann+wmf2_runoff_ann)

#Sea ice related stuff FRAZIL+MELT+SALT_F (Frazil + melt + brine)
wmt_ice_ann_diff = (wmt1_melt_ann+wmt1_frazil_ann+wmt1_salt_ann) - \
                   (wmt2_melt_ann+wmt2_frazil_ann+wmt2_salt_ann)
wmf_ice_ann_diff = (wmf1_melt_ann+wmf1_frazil_ann+wmf1_salt_ann) - \
                   (wmf2_melt_ann+wmf2_frazil_ann+wmf2_salt_ann)


### Plot the results

Plots of the **transformation** for total and total heat and total freshwater components

In [ ]:
#Plot transformations averaged over all time
f=plt.figure()
plt.plot(wmt1_ann.sigma,wmt1_ann/1e6,label="All "+label1,color='black',linestyle='-', linewidth=1)
plt.plot(wmt1_fresh_ann.sigma,wmt1_fresh_ann/1e6,label="Fresh "+label1,color='blue',linestyle='-', linewidth=1)
plt.plot(wmt1_heat_ann.sigma,wmt1_heat_ann/1e6,label="Heat "+label1,color='red',linestyle='-', linewidth=1)
plt.plot(wmt2_ann.sigma,wmt2_ann/1e6,label="All "+label2,color='black',linestyle=':', linewidth=1)
plt.plot(wmt2_fresh_ann.sigma,wmt2_fresh_ann/1e6,label="Fresh "+label2,color='blue',linestyle=':', linewidth=1)
plt.plot(wmt2_heat_ann.sigma,wmt2_heat_ann/1e6,label="Heat "+label2,color='red',linestyle=':', linewidth=1)
plt.title('Annual mean water mass transformation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Transformation Rate (Sv)')
plt.ylim([-25,30])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend()
plt.savefig('coastal_WMT_'+label1+'_and_'+label2+'_sigma2.png', bbox_inches='tight', dpi=200)

In [ ]:
colors = ['red' if (value > 0) else 'blue' for value in wmt_ann_diff]

f=plt.figure()
plt.bar(wmt_ann_diff.sigma,wmt_ann_diff/1e6,align='edge',edgecolor='black',color=colors,width=0.1)
plt.title('Annual mean water mass transformation difference: '+label1+'-'+label2)
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Total Transformation Rate (Sv)')
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.savefig('coastal_WMT_diff_'+label1+'_minus_'+label2+'_sigma2.png', bbox_inches='tight', dpi=200)

Plots of the **formation** for total and total heat and total freshwater components

In [ ]:
#Plot formation averaged over all time
f=plt.figure()
plt.plot(wmf1_ann.sigma,wmf1_ann/1e6,label="All "+label1,color='black',linestyle='-', linewidth=1)
plt.plot(wmf1_fresh_ann.sigma,wmf1_fresh_ann/1e6,label="Fresh "+label1,color='blue',linestyle='-', linewidth=1)
plt.plot(wmf1_heat_ann.sigma,wmf1_heat_ann/1e6,label="Heat "+label1,color='red',linestyle='-', linewidth=1)
plt.plot(wmf2_ann.sigma,wmf2_ann/1e6,label="All "+label2,color='black',linestyle=':', linewidth=1)
plt.plot(wmf2_fresh_ann.sigma,wmf2_fresh_ann/1e6,label="Fresh "+label2,color='blue',linestyle=':', linewidth=1)
plt.plot(wmf2_heat_ann.sigma,wmf2_heat_ann/1e6,label="Heat "+label2,color='red',linestyle=':', linewidth=1)
plt.title('Annual mean water mass formation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Formation Rate (Sv)')
plt.ylim([-15,10])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend()
plt.savefig('coastal_WMF_'+label1+'_and_'+label2+'_sigma2.png', bbox_inches='tight', dpi=200)

In [ ]:
colors = ['red' if (value > 0) else 'blue' for value in wmf_ann_diff]

f=plt.figure()
plt.bar(wmf_ann_diff.sigma,wmf_ann_diff/1e6,align='edge',edgecolor='black',color=colors,width=0.1)
plt.title('Annual mean water mass formation difference: '+label1+'-'+label2)
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Total Formation Rate (Sv)')
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.savefig('coastal_WMF_diff_'+label1+'_minus_'+label2+'_sigma2.png', bbox_inches='tight', dpi=200)

Plots of the **transformation** freshwater components

In [ ]:
#Plot transformation components averaged over all time
f=plt.figure()
plt.plot(wmt1_fresh_ann.sigma,wmt1_fresh_ann/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmt1_recons_ann.sigma,wmt1_recons_ann/1e6,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmt1_salt_ann.sigma,wmt1_salt_ann/1e6,label="Brine",color='red',linestyle='-', linewidth=1)
plt.plot(wmt1_frazil_ann.sigma,wmt1_frazil_ann/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmt1_evap_ann.sigma,wmt1_evap_ann/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmt1_melt_ann.sigma,wmt1_melt_ann/1e6,label="Ice melt",color='cyan',linestyle='-', linewidth=1)
plt.plot(wmt1_precip_ann.sigma,wmt1_precip_ann/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmt1_runoff_ann.sigma,wmt1_runoff_ann/1e6,label="Runoff",color='purple',linestyle='-', linewidth=1)

plt.title(label1+' annual mean water mass transformation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Transformation Rate (Sv)')
plt.ylim([-15,15])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend(loc=2,ncol=4)
plt.savefig('coastal_WMT_fresh_components_'+label1+'_sigma2.png', bbox_inches='tight', dpi=200)

In [ ]:
#Plot transformation components averaged over all time
f=plt.figure()
plt.plot(wmt2_fresh_ann.sigma,wmt2_fresh_ann/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmt2_recons_ann.sigma,wmt2_recons_ann/1e6,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmt2_salt_ann.sigma,wmt2_salt_ann/1e6,label="Brine",color='red',linestyle='-', linewidth=1)
plt.plot(wmt2_frazil_ann.sigma,wmt2_frazil_ann/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmt2_evap_ann.sigma,wmt2_evap_ann/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmt2_melt_ann.sigma,wmt2_melt_ann/1e6,label="Ice melt",color='cyan',linestyle='-', linewidth=1)
plt.plot(wmt2_precip_ann.sigma,wmt2_precip_ann/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmt2_runoff_ann.sigma,wmt2_runoff_ann/1e6,label="Runoff",color='purple',linestyle='-', linewidth=1)

plt.title(label2+' annual mean water mass transformation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Transformation Rate (Sv)')
plt.ylim([-15,15])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend(loc=2,ncol=4)
plt.savefig('coastal_WMT_fresh_components_'+label2+'_sigma2.png', bbox_inches='tight', dpi=200)

In [ ]:
#Plot transformation components averaged over all time; MUSHY - BL99
f=plt.figure()
plt.plot(wmt2_fresh_ann.sigma,(wmt1_fresh_ann-wmt2_fresh_ann)/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmt2_recons_ann.sigma,(wmt1_recons_ann-wmt2_recons_ann)/1e6 ,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmt2_salt_ann.sigma,(wmt1_salt_ann-wmt2_salt_ann)/1e6,label="Salt",color='red',linestyle='-', linewidth=1)
plt.plot(wmt2_frazil_ann.sigma,(wmt1_frazil_ann-wmt2_frazil_ann)/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmt2_evap_ann.sigma,(wmt1_evap_ann-wmt2_evap_ann)/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmt2_melt_ann.sigma,(wmt1_melt_ann-wmt2_melt_ann)/1e6,label="Melt",color='cyan',linestyle='-', linewidth=1)
plt.plot(wmt2_precip_ann.sigma,(wmt1_precip_ann-wmt2_precip_ann)/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmt2_runoff_ann.sigma,(wmt1_runoff_ann-wmt2_runoff_ann)/1e6,label="Roff",color='purple',linestyle='-', linewidth=1)


plt.title(label1+'-'+label2+' annual mean water mass transformation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,38])
plt.ylabel('Transformation Rate (Sv)')
plt.ylim([-2.5,2])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend(loc=4,ncol=4)
plt.savefig('coastal_WMT_fresh_components_diff_'+label1+'-'+label2+'.png', bbox_inches='tight', dpi=200)

Plots of the **formation** freshwater components

In [ ]:
#Plot formation components averaged over all time
f=plt.figure()
plt.plot(wmf1_fresh_ann.sigma,wmf1_fresh_ann/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmf1_recons_ann.sigma,wmf1_recons_ann/1e6,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmf1_salt_ann.sigma,wmf1_salt_ann/1e6,label="Salt",color='red',linestyle='-', linewidth=1)
plt.plot(wmf1_frazil_ann.sigma,wmf1_frazil_ann/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmf1_evap_ann.sigma,wmf1_evap_ann/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmf1_precip_ann.sigma,wmf1_precip_ann/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmf1_runoff_ann.sigma,wmf1_runoff_ann/1e6,label="Roff",color='purple',linestyle='-', linewidth=1)
plt.plot(wmf1_melt_ann.sigma,wmf1_melt_ann/1e6,label="Melt",color='cyan',linestyle='-', linewidth=1)

plt.title(label1+' annual mean water mass formation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Formation Rate (Sv)')
plt.ylim([-10,5])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend()
plt.savefig('coastal_WMF_components_'+label1+'.png', bbox_inches='tight', dpi=200)

In [ ]:
#Plot formation components averaged over all time
f=plt.figure()
plt.plot(wmf2_fresh_ann.sigma,wmf2_fresh_ann/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmf2_recons_ann.sigma,wmf2_recons_ann/1e6,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmf2_salt_ann.sigma,wmf2_salt_ann/1e6,label="Salt",color='red',linestyle='-', linewidth=1)
plt.plot(wmf2_frazil_ann.sigma,wmf2_frazil_ann/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmf2_evap_ann.sigma,wmf2_evap_ann/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmf2_precip_ann.sigma,wmf2_precip_ann/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmf2_runoff_ann.sigma,wmf2_runoff_ann/1e6,label="Roff",color='purple',linestyle='-', linewidth=1)
plt.plot(wmf2_melt_ann.sigma,wmf2_melt_ann/1e6,label="Melt",color='cyan',linestyle='-', linewidth=1)

plt.title(label2+' annual mean water mass formation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Formation Rate (Sv)')
plt.ylim([-10,5])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.legend()
plt.savefig('coastal_WMF_components_'+label2+'.png', bbox_inches='tight', dpi=200)

In [ ]:
#Plot formation components averaged over all time; MUSHY - BL99
f=plt.figure()
plt.plot(wmf2_fresh_ann.sigma,(wmf1_fresh_ann-wmf2_fresh_ann)/1e6,label="TotFresh",color='black',linestyle='-', linewidth=1)
#plt.plot(wmf2_recons_ann.sigma,(wmf1_recons_ann-wmf2_recons_ann)/1e6 ,label="Recons",color='grey',linestyle='-', linewidth=1)
plt.plot(wmf2_salt_ann.sigma,(wmf1_salt_ann-wmf2_salt_ann)/1e6,label="Salt",color='red',linestyle='-', linewidth=1)
plt.plot(wmf2_frazil_ann.sigma,(wmf1_frazil_ann-wmf2_frazil_ann)/1e6,label="Frazil",color='goldenrod',linestyle='-', linewidth=1)
plt.plot(wmf2_evap_ann.sigma,(wmf1_evap_ann-wmf2_evap_ann)/1e6,label="Evap",color='green',linestyle='-', linewidth=1)
plt.plot(wmf2_precip_ann.sigma,(wmf1_precip_ann-wmf2_precip_ann)/1e6,label="Precip",color='blue',linestyle='-', linewidth=1)
plt.plot(wmf2_runoff_ann.sigma,(wmf1_runoff_ann-wmf2_runoff_ann)/1e6,label="Roff",color='purple',linestyle='-', linewidth=1)
plt.plot(wmf2_melt_ann.sigma,(wmf1_melt_ann-wmf2_melt_ann)/1e6,label="Melt",color='cyan',linestyle='-', linewidth=1)

plt.title(label1+'-'+label2+' annual mean water mass formation')
plt.xlabel('sigma2 (kg m$^3$)')
plt.xlim([35,37.75])
plt.ylabel('Transformation Rate (Sv)')
plt.ylim([-2,2])
plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
plt.axvline(x=27.0, color='grey', linestyle='-',linewidth=0.5)
plt.axvline(x=27.5, color='grey', linestyle='-',linewidth=0.5)
plt.axvline(x=28.0, color='grey', linestyle='-',linewidth=0.5)
plt.axvline(x=28.2, color='grey', linestyle='-',linewidth=0.5)
plt.legend(loc=2,ncol=4)
plt.savefig('coastal_WMF_fresh_components_diff_'+label1+'-'+label2+'.png', bbox_inches='tight', dpi=200)

The largest two cyan formation wiggles (melt_f) fall: 
1) A positive peak centered at 37.0, which probably corresponds to the strengthening in the denser part of the SO overturning cell
2) A negative peak centered at 36.4, which probably corresponds to the weakening of the lighter half of the SO overturning cell. 



Plots of the transformation and formation **differences** due to processes

In [ ]:
# Plot differences in transformation/formation due to FReshwater and heat so we can attribute differences at different
# density categories
things_to_plot={}
things_to_plot['total transformation']=wmt_ann_diff
things_to_plot['total formation']=wmf_ann_diff
things_to_plot['transformation due to FW']=wmt_fresh_ann_diff
things_to_plot['FW formation due to FW']=wmf_fresh_ann_diff
things_to_plot['transformation due to heat']=wmt_heat_ann_diff
things_to_plot['formation due to heat']=wmf_heat_ann_diff

for wm in things_to_plot.keys():

    plot_me = things_to_plot[wm]
    colors = ['red' if (value > 0) else 'blue' for value in plot_me]

    f=plt.figure()
    
    plt.bar(plot_me.sigma,plot_me/1e6,align='edge',edgecolor='black',color=colors,width=0.1)
    #plt.title('Annual mean water mass transformation difference: '+label1+'-'+label2)
    plt.xlabel('sigma2 (kg m$^3$)')
    plt.xlim([35,37.75])
    plt.ylabel(wm+' (Sv)')
    plt.axhline(y=0.0, color='grey', linestyle='-', linewidth=0.5)
    plt.axvline(x=27.0, color='grey', linestyle='-',linewidth=0.5)
    plt.axvline(x=27.5, color='grey', linestyle='-',linewidth=0.5)
    plt.axvline(x=28.0, color='grey', linestyle='-',linewidth=0.5)
    plt.axvline(x=28.2, color='grey', linestyle='-',linewidth=0.5)
    plt.ylim([-2,1.5])


In [ ]:
#transformation by processes: 
#Total
#all heating
#E-P-R
#Sea ice related stuff: Frazil+Brine+Melt

f=plt.figure(figsize=(12,3))

things_to_plot ={}
things_to_plot['total'] = wmt_ann_diff
things_to_plot['heat'] = wmt_heat_ann_diff
things_to_plot['Evap-Precip-Runoff'] = wmt_epr_ann_diff
things_to_plot['sea ice (frazil+brine+melt)'] = wmt_ice_ann_diff

colors=['black','red','goldenrod','blue']

for ii,key in enumerate(things_to_plot.keys()):
    plot_me = things_to_plot[key]
    plt.bar(plot_me.sigma+0.015*(ii-2),plot_me/1e6,align='edge',edgecolor='None',\
            color=colors[ii],width=0.015, label = key)
    
plt.legend(loc=4,ncol = 2)
plt.xticks(plot_me.sigma)
plt.xlim([35,37.75])
plt.xlabel('sigma2 (kg m$^{-3}$)')
plt.ylabel('Transformation (Sv)')
plt.title('MUSHY-BL99 annual mean transformation difference by processes')
plt.axhline(0,color='dimgray',linewidth=1)

plt.savefig('coastal_WMT_processes_diff_'+label1+'-'+label2+'.png', bbox_inches='tight', dpi=200)

In [ ]:
#formation by processes: 
#Total
#all heating
#E-P-R
#Sea ice related stuff: Frazil+Brine+Melt

f=plt.figure(figsize=(12,3))

things_to_plot ={}
things_to_plot['total'] = wmf_ann_diff
things_to_plot['heat'] = wmf_heat_ann_diff
things_to_plot['Evap-Precip-Runoff'] = wmf_epr_ann_diff
things_to_plot['sea ice (frazil+brine+melt)'] = wmf_ice_ann_diff

colors=['black','red','goldenrod','blue']

for ii,key in enumerate(things_to_plot.keys()):
    plot_me = things_to_plot[key]
    plt.bar(plot_me.sigma+0.015*(ii-2),plot_me/1e6,align='edge',edgecolor='None',\
            color=colors[ii],width=0.015, label = key)
    
plt.legend(loc=4,ncol = 2)
plt.xticks(plot_me.sigma)
plt.xlim([35,37.75])
plt.xlabel('sigma2 (kg m$^{-3}$)')
plt.ylabel('Formation (Sv)')
plt.title('MUSHY-BL99 annual mean formation difference by processes')
plt.axhline(0,color='dimgray',linewidth=1)

plt.savefig('coastal_WMF_diff_'+label1+'-'+label2+'.png', bbox_inches='tight', dpi=200)